In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!pip uninstall fastai -y
!pip install fastai -q

import fastai
fastai.__version__

Uninstalling fastai-1.0.61:
  Successfully uninstalled fastai-1.0.61
     |████████████████████████████████| 194kB 6.2MB/s 
     |████████████████████████████████| 61kB 4.7MB/s 


'2.2.5'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title old dataset
# import os
# import shutil
# import time
# from google_drive_downloader import GoogleDriveDownloader as gdd

# if not os.path.exists("/content/images/valid.zip"):
#     # https://drive.google.com/file/d/1bXD5hR5WlIB6LsqDHmNn29EMA43Z82f3/view?usp=sharing
#     gdd.download_file_from_google_drive(file_id='1bXD5hR5WlIB6LsqDHmNn29EMA43Z82f3',
#                                         dest_path='./images/valid.zip',
#                                         unzip=True,
#                                         showsize=True,
#                                         )
#     # !rm -rf /content/images/valid.zip

# # download scripts

# !rm -rf *.py
# !rm -rf *.zip
# time.sleep(1)
# !wget -q https://github.com/veb-101/GAN-Colorisation/archive/master.zip
# !unzip -qq -o /content/master.zip 
# !mv ./GAN-Colorisation-master/trainer.py ./trainer.py
# !mv ./GAN-Colorisation-master/utils.py ./utils.py
# !mv ./GAN-Colorisation-master/model.py ./model.py
# !mv ./GAN-Colorisation-master/data_loader.py ./data_loader.py
# !rm -rf /content/GAN-Colorisation-master



# if not os.path.exists("/content/images/train.zip"):
#     # https://drive.google.com/file/d/1c5WQwglbVL9S_LHH5E9XU9bCWLzZg_V7/view?usp=sharing
#     gdd.download_file_from_google_drive(file_id='1c5WQwglbVL9S_LHH5E9XU9bCWLzZg_V7',
#                                         dest_path='./images/train.zip',
#                                         unzip=True,
#                                         showsize=True,
#                                         )
#     # !rm -rf /content/images/train.zip

In [4]:
#@title Download datasets and scripts

import os
import shutil
import time
from google_drive_downloader import GoogleDriveDownloader as gdd



!rm -rf *.py
time.sleep(1)
!wget -q https://github.com/veb-101/GAN-Colorisation/archive/master.zip
!unzip -qq -o /content/master.zip 
!mv ./GAN-Colorisation-master/trainer.py ./trainer.py
!mv ./GAN-Colorisation-master/utils.py ./utils.py
!mv ./GAN-Colorisation-master/model.py ./model.py
!mv ./GAN-Colorisation-master/data_loader.py ./data_loader.py
!rm -rf /content/GAN-Colorisation-master
!rm -f master.zip*


# https://drive.google.com/file/d/1rDE8V7FuvTtyhxJ1MyJUrMeYvvoG0gCT/view?usp=sharing
if not os.path.exists("images.zip"):
    gdd.download_file_from_google_drive(file_id='1rDE8V7FuvTtyhxJ1MyJUrMeYvvoG0gCT',
                                        dest_path='./images.zip',
                                        unzip=True,
                                        showsize=True,
                                        )

1.1 GiB Done.


In [33]:
#@title setup

import os
import gc
import random
import shutil
import warnings
import importlib
import numpy as np
# from tqdm. import tqdm
from tqdm.notebook import tqdm



import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler

import model
import utils
import trainer
import data_loader

seed = 41
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

importlib.reload(data_loader)
importlib.reload(utils)
importlib.reload(trainer)
importlib.reload(model)

from trainer import MainModel
from utils import init_model
from data_loader import make_dataloaders
from utils import update_losses, visualize
from utils import init_model, AverageMeter
from utils import create_loss_meters, log_results
from model import Generator_Res_Unet, Discriminator, Generator_Unet



def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")

device = get_default_device()

warnings.simplefilter("ignore", UserWarning)

In [34]:
#@title Load Model
model_type = "Generator_Res_Unet-pretrained" #@param ["Generator_Res_Unet-pretrained", "Generator_Res_Unet-new", "Generator_Unet"]

if model_type == "Generator_Res_Unet-pretrained":
    opt=1
    model = Generator_Res_Unet()
    pretrained = True

elif model_type == "Generator_Res_Unet-new":
    opt=2
    model = Generator_Res_Unet()
    pretrained = False
elif model_type == "Generator_Unet":
    opt=3
    model_3 = Generator_Unet()


def load_model(config, ckpt_number=-1, device=None, prefix="warm_"):
    
    lr_G = config["lr_G"]
    lr_D = config["lr_D"]
    beta1 = config["beta1"]
    beta2 = config["beta2"]
    load_previous = config["load_previous"]
    
    epoch = 0
    if opt in (1, 2,):
        net_G = init_model(model.get_model(pretrained=pretrained), device)
    else:
        net_G = init_model(model_3, device)

    net_D = init_model(Discriminator(input_channels=3), device)
    scaler_G = GradScaler()
    scaler_D = GradScaler()

    opt_G = optim.Adam(net_G.parameters(), lr=lr_G, betas=(beta1, beta2))
    opt_D = optim.Adam(net_D.parameters(), lr=lr_D, betas=(beta1, beta2))

    drive_path = r"/content/drive/MyDrive/Project-Colorisation"
    print(f"[*] Finding checkpoint {ckpt_number} in {drive_path}")

    checkpoint_file = f"{prefix}checkpoint_{ckpt_number}.tar"
    checkpoint_path = os.path.join(drive_path, checkpoint_file)

    if not os.path.exists(checkpoint_path):
        print(f"[!] No checkpoint for epoch {ckpt_number}")
    else:    
        checkpoint = torch.load(checkpoint_path)

        epoch = checkpoint["epoch"]

        net_G.load_state_dict(checkpoint[f"net_G"])
        print("Generator weight loaded")

        try:
            net_D.load_state_dict(checkpoint[f"net_D"])
            print("Discriminator weights loaded.")
        except:
            pass

        if load_previous:
            try:  
                opt_D.load_state_dict(checkpoint[f"opt_D"])
                print("Discriminator optimizer loaded.")
            except:
                pass

            opt_G.load_state_dict(checkpoint[f"opt_G"])
            print("Optimizer's state loaded")
            
            try:
                scaler_G.load_state_dict(checkpoint[f"scaler_G"])
                print("Grad Scaler - Generator loaded")
                scaler_D.load_state_dict(checkpoint[f"scaler_D"])
                print("Grad Scaler - Discriminator loaded")
            except Exception as e:
                pass
        

    return_ = {
        "epoch": epoch,
        "net_G": net_G,
        "opt_G": opt_G,
        "scaler_G": scaler_G,
        "net_D": net_D,
        "scaler_D": scaler_D,
        "opt_D": opt_D
    }
    return return_

## Pretrain Generator

In [51]:
def pretrain_generator(train_dl, valid_dl, net, scaler, opt, criterion, start_epoch, epochs, device=None):
    savedir_train = "/content/warmup_train/train"
    savedir_val = "/content/warmup_train/validation"
    drive_path = "/content/drive/MyDrive/Project-Colorisation"
    os.makedirs(savedir_train, exist_ok=True)
    os.makedirs(savedir_val, exist_ok=True)
    os.makedirs(drive_path, exist_ok=True)

    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
        opt,
        [10, 20, 30]  
    )
    
    for e in range(start_epoch, start_epoch+epochs):
        loss_meter = AverageMeter()
        psnr_val = []
        ssim_val = []
        training_loader_iter = iter(train_dl)
        length_train = len(training_loader_iter)
        net.train()
        
        torch.cuda.empty_cache()
        gc.collect()
        for i in tqdm(range(length_train), desc=f"Epoch: [{e}/{start_epoch+epochs-1}]"):
            data = next(training_loader_iter)

            opt.zero_grad()
            L, ab = data['L'].to(device), data['ab'].to(device)

            with autocast():
                preds = net(L)
                loss = criterion(preds, ab)
            
            scaler.scale(loss).backward()
            scaler.step(opt)
            scale_gen = scaler.get_scale()
            scaler.update()
            skip_lr_sched = scale_gen != scaler.get_scale()

            # loss.backward()
            # opt.step()
            loss_meter.update(loss.detach().item(), L.size(0))
            torch.cuda.empty_cache()
            gc.collect()
            
            if i == 0 or i == int(length_train / 2) or i == (length_train - 1):
                print(f"Iteration:[{i+1}/{length_train}] L1 Loss: [{loss_meter.avg:.5f}]")
                _, _ = visualize(net, 
                        data, 
                        save_name=os.path.join(savedir_train, f"Train_Epoch_{e}-{i}.png"), 
                        device=device)
            

        
        
        for idx, valid_data in enumerate(valid_dl):
            psnr_batch, ssim_batch_ = visualize(net, 
                                                valid_data, 
                                                save_name=os.path.join(savedir_val, f"Validation_Epoch_{e}-{idx}.png"), 
                                                device=device)
            
            psnr_val.append(psnr_batch)
            ssim_val.append(ssim_batch_)
        
        print(f"Validation PSNR: {sum(psnr_val)/len(psnr_val):.5}, SSIM: {sum(ssim_val)/len(ssim_val):.5}")
        
        if not skip_lr_sched:
            lr_scheduler.step()
        
        file_name = f"warm_checkpoint_{e}.tar"
        torch.save({
            "epoch": e,
            "net_G": net.state_dict(),
            "opt_G": opt.state_dict(),
            "scaler_G": scaler.state_dict(),
        }, file_name)
        
        shutil.copyfile(file_name, os.path.join(drive_path, file_name))

        remove_file_name = f"/content/warm_checkpoint_{e-1}.tar"
        remove_drive_file_name = f"/content/drive/MyDrive/Project-Colorisation/warm_checkpoint_{e-1}.tar"
        try:
          os.remove(remove_file_name)
          os.remove(remove_drive_file_name)
        except:
          pass

## Train Gan

In [52]:
def train_model(model, train_dl, valid_dl, start_epoch, epochs):
    savedir_train = "/content/gan_train/train"
    savedir_val = "/content/gan_train/validation"
    drive_path = "/content/drive/MyDrive/Project-Colorisation"
    os.makedirs(savedir_train, exist_ok=True)
    os.makedirs(savedir_val, exist_ok=True)
    os.makedirs(drive_path, exist_ok=True)

    for e in range(start_epoch, start_epoch+epochs):
        loss_meter_dict = create_loss_meters()
        psnr_val = []
        ssim_val = []
        training_loader_iter = iter(train_dl)
        length_train = len(training_loader_iter)
        
        for i in tqdm(range(length_train), desc=f"Epoch: [{e}/{start_epoch+epochs-1}]"):
            data = next(training_loader_iter)
            model.setup_input(data)
            model.optimize()
            
            # function updating the log objects
            update_losses(model, loss_meter_dict, count=data["L"].size(0)) 
            
            if i == 0 or i == int(length_train / 2) or i == (length_train - 1):
                print(f"Iteration:[{i+1}/{length_train}]", end=" ")
                # function to print out the losses
                log_results(loss_meter_dict)  
                _, _ = visualize(net, 
                        data, 
                        save_name=os.path.join(savedir_train, f"Train_Epoch_{e}-{i}.png"), 
                        device=device)
            
        for idx, valid_data in enumerate(valid_dl):
            psnr_batch, ssim_batch_ = visualize(net, 
                                                valid_data, 
                                                save_name=os.path.join(savedir_val, f"Validation_Epoch_{e}-{idx}.png"), 
                                                device=device)
            
            psnr_val.append(psnr_batch)
            ssim_val.append(ssim_batch_)
        
        print(f"Validation PSNR: {sum(psnr_val)/len(psnr_val):.5}, SSIM: {sum(ssim_val)/len(ssim_val):.5}")


        file_name = f"gan_checkpoint_{e}.tar"
        torch.save({
            "epoch": e,
            "net_G": model.net_G.state_dict(),
            "opt_G": model.opt_G.state_dict(),
            "scaler_G": model.scaler_G.state_dict(),
            "net_D": model.net_D.state_dict(),
            "opt_D": model.opt_D.state_dict(),
            "scaler_D": model.scaler_D.state_dict(),
        }, file_name)

        remove_file_name = f"/content/gan_checkpoint_{e-1}.tar"
        remove_drive_file_name = f"/content/drive/MyDrive/Project-Colorisation/gan_checkpoint_{e-1}.tar"
        try:
          os.remove(remove_file_name)
          os.remove(remove_drive_file_name)
        except:
          pass

## Training Setup

In [53]:
#@title Dataloaders 
num_images_train =  -1#@param {type:"integer"}
num_images_valid =  -1#@param {type:"integer"}


if __name__ == "__main__":
    train_ = "/content/train"
    valid_ = "/content/valid"
    device = get_default_device()
    train_dl = make_dataloaders(path=train_, num_images=num_images_train)
    valid_dl = make_dataloaders(path=valid_, num_images=num_images_valid, 
                                is_training=False, shuffle=False)
        
    print(f"Number of batches ::Train:: {len(train_dl)}, ::Valid:: {len(valid_dl)}")

Number of batches ::Train:: 1, ::Valid:: 1


In [54]:
config = {
    "lr_G": 1e-4,
    "lr_D": 2e-4,
    "beta1": 0.5,
    "beta2": 0.999,
    "lambda_L1": 100.0,
    "load_previous": True
}

## Warmup generator

In [ ]:
# load previous epoch optimizers and grad scaler
config["load_previous"] = True # First run
# config["load_previous"] = True # uncomment for subsequent runs of same Type

load_ckpt = -1
gen_load = load_model(config, ckpt_number=load_ckpt, device=device, prefix="warm_")

net_G = gen_load["net_G"]
scaler_G = gen_load["scaler_G"]
opt_G = gen_load["opt_G"]
last_epoch = gen_load["epoch"]

criterion = nn.L1Loss()     
criterion.to(device)
start_epoch = last_epoch + 1
epochs = 50

pretrain_generator(train_dl, valid_dl, net_G, scaler_G, opt_G, criterion, start_epoch, epochs, device=device)

## Train GAN

In [ ]:
# train Gan model

config["lr_G"] = 2e-4
torch.cuda.empty_cache()
gc.collect()

# load previous epoch optimizers and grad scaler
config["load_previous"] = False # First run
# config["load_previous"] = True # uncomment for subsequent runs of same Type

load_ckpt = -1
gan_load = load_model(config, ckpt_number=load_ckpt, device=device, prefix="warm_")


# pretrained Generator 
net_G = gan_load["net_G"]
scaler_G = gan_load["scaler_G"]
opt_G = gan_load["opt_G"]

# newly Discriminator if load_previous = False else previously trained
net_D = gan_load["net_D"]
scaler_D = gan_load["scaler_D"]
opt_D = gan_load["opt_D"]

last_epoch = gan_load["epoch"] 

if not config["load_previous"]:
    last_epoch = 0

start_epoch = last_epoch + 1
epochs = 100

model = MainModel(net_G=net_G,
                  net_D=net_D,
                  scaler_G=scaler_G,
                  scaler_D=scaler_D,
                  opt_G=opt_G,
                  opt_D=opt_D,
                  device=device)

train_model(model, train_dl, valid_dl, start_epoch, epochs)

model initialized with norm initialization
model initialized with norm initialization
[*] Finding checkpoint 40 in /content/drive/MyDrive/Project-Colorisation
Generator weight loaded


Iteration:[1/311] loss_D: 0.96842 loss_G_GAN: 1.99699 loss_G_L1: 7.94816 loss_G: 9.94515 
Iteration:[156/311] loss_D: 0.63796 loss_G_GAN: 1.06942 loss_G_L1: 8.15902 loss_G: 9.22844 
Iteration:[311/311] loss_D: 0.58585 loss_G_GAN: 1.23246 loss_G_L1: 8.39303 loss_G: 9.62549 



Iteration:[1/311] loss_D: 0.48683 loss_G_GAN: 1.04264 loss_G_L1: 9.08921 loss_G: 10.13185 
Iteration:[156/311] loss_D: 0.54304 loss_G_GAN: 1.40505 loss_G_L1: 8.85972 loss_G: 10.26478 
Iteration:[311/311] loss_D: 0.54895 loss_G_GAN: 1.39509 loss_G_L1: 8.99037 loss_G: 10.38546 



Iteration:[1/311] loss_D: 1.06632 loss_G_GAN: 1.43154 loss_G_L1: 8.49142 loss_G: 9.92295 
Iteration:[156/311] loss_D: 0.55211 loss_G_GAN: 1.35546 loss_G_L1: 9.22525 loss_G: 10.58071 
Iteration:[311/311] loss_D: 0.55648 loss_G_GAN: 1.33690 loss_G_L1: 9.24526 loss_G: 10.58216 



Iteration:[1/311] loss_D: 1.10671 loss_G_GAN: 1.07637 loss_G_L1: 8.39686 loss_G: 9.47323 
Iteration:[156/311] loss_D: 0.56934 loss_G_GAN: 1.30211 loss_G_L1: 9.20352 loss_G: 10.50563 
Iteration:[311/311] loss_D: 0.56736 loss_G_GAN: 1.29202 loss_G_L1: 9.30356 loss_G: 10.59558 



...

Iteration:[1/311] loss_D: 0.46267 loss_G_GAN: 1.61251 loss_G_L1: 10.13006 loss_G: 11.74257 
Iteration:[156/311] loss_D: 0.58012 loss_G_GAN: 1.18372 loss_G_L1: 9.29232 loss_G: 10.47605 
Iteration:[311/311] loss_D: 0.58335 loss_G_GAN: 1.18488 loss_G_L1: 9.26880 loss_G: 10.45368 



Iteration:[1/311] loss_D: 1.27378 loss_G_GAN: 1.40837 loss_G_L1: 8.20723 loss_G: 9.61560 
Iteration:[156/311] loss_D: 0.58342 loss_G_GAN: 1.18556 loss_G_L1: 9.28871 loss_G: 10.47428 
Iteration:[311/311] loss_D: 0.58491 loss_G_GAN: 1.17676 loss_G_L1: 9.21356 loss_G: 10.39032 



Iteration:[1/311] loss_D: 0.44458 loss_G_GAN: 1.72184 loss_G_L1: 8.32979 loss_G: 10.05163 
Iteration:[156/311] loss_D: 0.58592 loss_G_GAN: 1.17442 loss_G_L1: 9.17469 loss_G: 10.34911 
Iteration:[311/311] loss_D: 0.58637 loss_G_GAN: 1.18365 loss_G_L1: 9.20319 loss_G: 10.38684 



Iteration:[1/311] loss_D: 0.69848 loss_G_GAN: 0.93517 loss_G_L1: 8.72106 loss_G: 9.65623 


KeyboardInterrupt: ignored

## Random


Run js function in console
```
function KeepClicking(){
console.log("Clicking");
document.querySelector("colab-connect-button").click()
}
setInterval(KeepClicking,300000)
```



In [ ]:
# !rm -rf /content/drive/MyDrive/Project-Colorisation/*tar
# !rm -rf *tar
# !rm -rf /content/gan_train
# !rm -rf /content/warmup_train